In [1]:
import tensorflow as tf
from tensorflow.python.keras import layers
import gensim
from tensorflow.python import keras
from tensorflow.python.keras import backend as K
import re
import numpy as np
import pandas as pd
import sklearn.metrics
from nltk.corpus import stopwords

In [2]:
import sys
sys.path.append("../scripts/")
import TweetToVect as TW
from importlib import reload
reload(TW)
TW.TweetToVect

TweetToVect.TweetToVect

In [3]:
with open("../models/tweet_to_vect_encoder_b.json", 'r') as f:
    loaded_model_encoder = tf.keras.models.model_from_json(f.read(), custom_objects={'TransformerBlock': TW.TransformerBlock, 'MultiHeadSelfAttention': TW.MultiHeadSelfAttention})
loaded_model_encoder.load_weights("../models/tweet_to_vect_encoder_b.h5")

loaded_word_model = gensim.models.fasttext.load_facebook_vectors('../models/words_fast_text.bin')

In [4]:
model = TW.TweetToVect()
model.load_encoder(model=loaded_model_encoder)
model.load_word_model(model=loaded_word_model)
model.tweet_length
print(model.get_vector('Hola!').shape)
# print(model.get_vectors(['Hola!']).shape)
# print(model.clean_text('Hola! esto si es un tweet'))

(150,)


In [ ]:
# model = TweetToVect()
# model.load_encoder('tweet_to_vect_encoder_a', '../models/')
# model.load_word_model('../models/words_fast_text.bin')

In [5]:
print(model.tweet_length, model.word_dim)

10 100


In [6]:
df = pd.read_pickle('../datasets/tweets_cleaned.pkl')
df = df[(df.lang=='es')&(df.favorite_count > 10)]
tokenize = lambda text: text.split(' ')
df.loc[:,'tweet_l'] = df.full_text_cleaned.apply(lambda text: len(tokenize(text)))

In [7]:
df_sample = df[df.tweet_l > 5].sample(30000)
tweet_vectors = model.get_vectors(df_sample.full_text_cleaned.values, True)

In [8]:
to_compare_texts = [
    """
        En una entrevista radial, el presidente @alferdez afirmó que "es una discusión falsa la cantidad de miembros que podría tener la Corte Suprema de Justicia" en la reforma judicial impulsada esta semana.
    """,
    """
        Recordá que para prevenir contagios de Covid-19 es fundamental respetar los protocolos de higiene y cuidado
    """,
    """
        A quienes justifican la necesidad de la reforma judicial x la inseguridad (robos y homicidios), les digo q la reforma oficialista se refiere a juzgados federales. El robo y el homicidio NO son competencia de los juzgados federales. Buscan impunidad no luchar contra la inseguridad
    """
]
to_compare = []
for i in to_compare_texts:
    to_compare.append(model.get_vector(i))
sklearn.metrics.pairwise.cosine_similarity(to_compare, to_compare)

array([[0.99999994, 0.39405197, 0.5768572 ],
       [0.39405197, 0.9999999 , 0.38733357],
       [0.5768572 , 0.38733357, 1.        ]], dtype=float32)

In [33]:
for i in range(30):
    to_compare = df[df.tweet_l > 7].sample(1)
    to_compare_cleaned = to_compare.full_text_cleaned.values
    to_compare_text = to_compare.full_text.values
    to_compare_vector = model.get_vectors(to_compare_cleaned, True)
    
    similarity_index, similarity_sorted = model.similar_from_target_vector(to_compare_vector, tweet_vectors)
    
    print('SIMILAR TO:', to_compare_text[0])
    similar_texts = df_sample.iloc[similarity_index].head(3).full_text.values
    for i in range(len(similar_texts)):
        print('----', round(similarity_sorted[i], 2), similar_texts[i])
    print('\n')

SIMILAR TO: #HOY acompañando al @MaipuMunicipio en la entrega de lavandina a sus vecinas y vecinos. 

Felicitaciones @MatiasStevanato y especialmente a todas las voluntarias y voluntarios por la gran tarea del "Programa Lavandina Municipal en casa"

La comunidad organizada es el camino https://t.co/fZxH5Ga7Xt
---- 0.67 En la entrega del premio "Julio Strassera" tuve el orgullo de entregar la distinción a @rgillavedra Agradezco por ello a @SilviaEdePerez https://t.co/LXFwkAE1Xj
---- 0.65 Se logró en cuatro días, hacer más de 6.400 test para analizar el impacto del brote. Y gracias a Dios, no se ha expandido de manera exponencial, y hasta ahora aparece contenido.
---- 0.64 📣| Acompañé a @MatiasStevanato en su Acto de Asunción como #IntendenteDeMaipú

Seguramente se seguirá desarrollando un departamento para todos y todas. ¡Gracias por la invitación y felicitaciones compañero! https://t.co/GqbLKXcfoc


SIMILAR TO: #Video ¡Ay, Diego! El Ruso Rodríguez y un blooper que decretó la derrota 1-

In [20]:
for i in ['🌈', '🛫', '💚', '👮‍♂️']:
    to_compare_text = [i+' '+i+' '+i]
    to_compare_vector = model.get_vectors(to_compare_text, True)
    
    similarity_index, similarity_sorted = model.similar_from_target_vector(to_compare_vector, tweet_vectors)
    
    print('SIMILAR TWEETS TO:', to_compare_text[0].replace('\n', ' '))
    similar_texts = df_sample.iloc[similarity_index].head(4).full_text.apply(lambda x: x.replace('\n', ' ')).values
    for i in range(len(similar_texts)):
        print('----', round(similarity_sorted[i], 2), similar_texts[i])
    print('-----------------------------\n')

SIMILAR TWEETS TO: 🌈 🌈 🌈
---- 0.85 🏳️‍🌈✨ buenas noches 🏳️‍🌈✨ https://t.co/EUEvc0pssB
---- 0.82 Felices 10 años de Matrimonio igualitario ❤🌈🌈🌈 https://t.co/6hSKdplA5K
---- 0.78 ¡Todos los días! 🏳️‍🌈  #IDAHOT https://t.co/kQ9JEFpfet
---- 0.73 28 de junio día Internacional del #Orgullo   •AMOR ES AMOR• 🏳️‍🌈  #InduatriasCreativasGC https://t.co/hLIIElxbY7
-----------------------------

SIMILAR TWEETS TO: 🛫 🛫 🛫
---- 0.66 Ya en El Palomar para ser parte del vuelo inaugural de @flybondioficial A Río de Janeiro!! #MeRioDeJaneiro https://t.co/cWr5X6JvH5
---- 0.64 #MartesDeAviones @flybondioficial tramitame esos vuelos apenas termine la Cuarentena. https://t.co/fVu8zBwtea
---- 0.64 Sexo virtual en pandemia por ⁦@muscarijoseok⁩ en ⁦@NAagencia⁩ @experienciasexx @sexvirtual2020 #sexvirtual  https://t.co/7d18L5DTFc
---- 0.62 Hoy es #MartesDeAviones!!!! @flybondioficial  #spottingdearchivo #quedatencasa #QuedateEnTuCasa  #nosonvacaciones  #spotter #spotting #aviones #aviationphoto #avgeeks #avgeek #c